In [ ]:
IMAGE_SIZE = (608, 608)

# Test Loss

In [ ]:
import torch
from src.loss import Loss
%load_ext autoreload
%autoreload 2

thm = torch.zeros((224, 224, 1), dtype=torch.float32)
gi = torch.tensor([30, 100, 150], dtype=torch.int64)
gj = torch.tensor([30, 50, 200], dtype=torch.int64)
offset = torch.tensor([[5, 10, 5, 10], [20, 10, 20, 10], [10, 15, 10, 15]], dtype=torch.int64)
Loss.draw_gaussian(thm, gi, gj, offset)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.imshow(thm[:, :, 0].cpu().numpy(), cmap="hot")
plt.show()

In [ ]:
(thm == 1).nonzero()

# Test Loss t2p, PostProcessor p2t

In [ ]:
import torch
from src.loss import Loss
from src.post_processor import PostProcessor
%load_ext autoreload
%autoreload 2

criterion = Loss(IMAGE_SIZE)
post_processor = PostProcessor(IMAGE_SIZE)

# Test Loss, PostProcessor

# Test Loss, PostProcessor, Accumulator, Trainer

In [ ]:
import numpy as np

class MockDataset(object):
    
    def __init__(self, length):
        super().__init__()
        self.length = length
        
    def __getitem__(self, idx):
        return {
            "image": np.empty((608, 608, 3), dtype=np.float32),
            "center": np.array([[50, 50]], dtype=np.float32),
            "offset": np.array([[10, 10, 10, 10]], dtype=np.float32)
        }
    
    def __len__(self,):
        return self.length

In [ ]:
from torch import nn
import torch

class MockModel(nn.Module):
    
    def __init__(self,):
        super().__init__()
        self.preds_hm = nn.Parameter(
            torch.ones((1, 1, IMAGE_SIZE[1] // 4, IMAGE_SIZE[0] // 4), dtype=torch.float32)
        )
        self.preds_wh = nn.Parameter(
            torch.ones((1, 4, IMAGE_SIZE[1] // 4, IMAGE_SIZE[0] // 4), dtype=torch.float32)
        )
        self.preds_reg = nn.Parameter(
            torch.ones((1, 2, IMAGE_SIZE[1] // 4, IMAGE_SIZE[0] // 4), dtype=torch.float32)
        )
        
    def forward(self, images):
        return [{"hm": torch.tile(self.preds_hm, (images.size(0), 1, 1, 1)),
                "wh": torch.tile(self.preds_wh, (images.size(0), 1, 1, 1)),
                "reg": torch.tile(self.preds_reg, (images.size(0), 1, 1, 1))}]

In [ ]:
from src.trainer import Trainer
%load_ext autoreload
%autoreload 2

net = MockModel()
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda",
    checkpoint_dir=None, model_path=None
)

In [ ]:
trainer.optimizer.param_groups[0]["lr"] = 1e-2
trainer.optimizer.param_groups[0]["weight_decay"] = 1e-4

In [ ]:
import tempfile
from pathlib import Path

with tempfile.TemporaryDirectory() as tmpdir:
    trainer.checkpoint_dir = Path(tmpdir)
    trainer.fit(MockDataset(1000), MockDataset(1), batch_size=4, epochs=20)

In [ ]:
net.eval()
preds = (net.preds_hm, net.preds_wh, net.preds_reg)
preds_hm = trainer.pre_detector(net.preds_hm)
preds_center, preds_offset, preds_score = trainer.post_processor(
    preds_hm.detach().cpu().numpy(), net.preds_wh.detach().cpu().numpy(),
    net.preds_reg.detach().cpu().numpy()
)
preds_center, preds_offset, preds_score